## Cryptocurrency trading bot.

### Install if needed.

In [ ]:
##%pip install python-binance==0.7.5
#%pip install --upgrade python-binance
#%pip install paramiko

### Prelude.

In [ ]:
# Time prelude for optimization purposes.
import time
t1 = time.time()

# Import libraries.
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import get_timezone_offset_in_seconds
from cryptocurrency.conversion_table import get_conversion_table
from cryptocurrency.trader.wallet import select_asset_with_biggest_wallet

# Manage API keys.
authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client

# Get all available pair information for trading.
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info

# Precalculate UTC offset for inter-server communication coherence.
offset_s = get_timezone_offset_in_seconds()

# Precalculate conversion_table.
conversion_table = get_conversion_table(client=client, 
                                        exchange_info=exchange_info, 
                                        offset_s=offset_s, 
                                        dump_raw=False, 
                                        as_pair=True, 
                                        minimal=False, 
                                        extra_minimal=False, 
                                        super_extra_minimal=False, 
                                        convert_to_USDT=False)

# Get highest USDT-converted held asset and determine priority.
from_asset, converted_quantity, quantity, priority = \
    select_asset_with_biggest_wallet(client=client, 
                                     conversion_table=conversion_table, 
                                     exchange_info=exchange_info)

# Display how long the prelude took.
t2 = time.time()
print('Elapsed time:', t2 - t1, 'seconds.')

# Display wallet contents.
from_asset, converted_quantity, quantity, priority

### Tests.

In [ ]:
import pandas as pd

def add_entry_to_blacklist(blacklist, pair, conversion_table, exchange_info, reason='stop_loss'):
    base_asset_from_pair = exchange_info[exchange_info['symbol'] == pair]['base_asset'].iat[0]
    if base_asset_from_pair not in blacklist['base_asset'].tolist():
        new_blacklist_entry = conversion_table[conversion_table['symbol'] == pair][['symbol', 'close']].copy()
        new_blacklist_entry['base_asset'] = base_asset_from_pair
        new_blacklist_entry['take_profit_count'] = 0
        new_blacklist_entry['stop_loss_count'] = 0
        new_blacklist_entry = new_blacklist_entry[['symbol', 'base_asset', 'close', 
                                                   'take_profit_count', 'stop_loss_count']]
        blacklist = pd.concat([blacklist, new_blacklist_entry], axis='index')
    else:
        pair = blacklist[blacklist['base_asset'] == base_asset_from_pair]['symbol'].iat[0]
        new_blacklist_entry = conversion_table[conversion_table['symbol'] == pair][['symbol', 'close']].copy()
        new_blacklist_entry['base_asset'] = base_asset_from_pair
        new_blacklist_entry['take_profit_count'] = \
            blacklist[blacklist['base_asset'] == base_asset_from_pair]['take_profit_count'].iat[0]
        new_blacklist_entry['stop_loss_count'] = \
            blacklist[blacklist['base_asset'] == base_asset_from_pair]['stop_loss_count'].iat[0]
        new_blacklist_entry = new_blacklist_entry[['symbol', 'base_asset', 'close', 
                                                   'take_profit_count', 'stop_loss_count']]
        blacklist = pd.concat([blacklist, new_blacklist_entry], axis='index')
        blacklist = blacklist.drop_duplicates(subset=['base_asset'], keep='last')
    pair = blacklist[blacklist['base_asset'] == base_asset_from_pair]['symbol'].iat[0]
    if reason == 'take_profit':
        blacklist.loc[blacklist['symbol'] == pair,'take_profit_count'] += 1
    if reason == 'stop_loss':
        blacklist.loc[blacklist['symbol'] == pair,'stop_loss_count'] += 1
    return blacklist

def check_if_asset_from_pair_is_buyable(blacklist, pair, exchange_info, 
                                        take_profit_count=6, stop_loss_count=3):
    base_asset_from_pair = exchange_info[exchange_info['symbol'] == pair]['base_asset'].iat[0]
    is_buyable = True
    if base_asset_from_pair in blacklist['base_asset'].tolist():
        pair = blacklist[blacklist['base_asset'] == base_asset_from_pair]['symbol'].iat[0]
        if blacklist.loc[blacklist['symbol'] == pair,'take_profit_count'] >= take_profit_count:
            is_buyable = False
        if blacklist.loc[blacklist['symbol'] == pair,'stop_loss_count'] >= stop_loss_count:
            is_buyable = False
    return is_buyable

def check_take_profit_and_stop_loss(blacklist, from_asset, to_asset, conversion_table, exchange_info, 
                                    latest_asset, take_profit=None, stop_loss=None):
    if from_asset in blacklist['base_asset'].tolist():
        pair = blacklist[blacklist['base_asset'] == from_asset]['symbol'].iat[0]
        price_when_bought = blacklist[blacklist['base_asset'] == from_asset]['close'].iat[0]
        price_now = conversion_table[conversion_table['symbol'] == pair]['close'].iat[0]
        percent_gain = ((price_now - price_when_bought) / price_when_bought) * 100.0
        if take_profit is not None:
            if percent_gain >= take_profit:
                to_asset = latest_asset
                blacklist = add_entry_to_blacklist(blacklist, pair, exchange_info, reason='take_profit')
        if stop_loss is not None:
            if percent_gain <= -stop_loss:
                to_asset = latest_asset
                blacklist = add_entry_to_blacklist(blacklist, pair, exchange_info, reason='stop_loss')
    return blacklist, to_asset

def remove_older_entries_in_blacklist(blacklist, frequency='15min'):
    frequency = pd.tseries.frequencies.to_offset(frequency)
    return blacklist.loc[(pd.Timestamp.utcnow().tz_localize(None) - blacklist.index) < frequency]

blacklist = pd.DataFrame(columns=['symbol', 'base_asset', 'close', 'take_profit_count', 'stop_loss_count'])

conversion_table = get_conversion_table(client=client, 
                                        exchange_info=exchange_info, 
                                        offset_s=offset_s, 
                                        dump_raw=False, 
                                        as_pair=True, 
                                        minimal=False, 
                                        extra_minimal=False, 
                                        super_extra_minimal=False, 
                                        convert_to_USDT=False)

pair = 'BTCUSDT'
blacklist = add_entry_to_blacklist(blacklist, pair, conversion_table, 
                                   exchange_info, reason=None)

blacklist = remove_older_entries_in_blacklist(blacklist, frequency='15min')
blacklist

### Main loop buying screened assets in pairs which are connected to the wallet holdings.

In [ ]:
sell_asset = 'BUSD'
to_asset = sell_asset
take_profit = 10.0
stop_loss = None
keys_file = 'server_keys.txt'
input_log = '~/workspace/crypto_logs/crypto_input_log_15s.txt'
output_log_screened = '~/workspace/crypto_logs/crypto_output_log_1d_screened.txt'

from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import select_pair_with_highest_quote_volume_from_base_asset
from cryptocurrency.trader.ssh import Ssh
from cryptocurrency.trader.wallet import select_asset_with_biggest_wallet
from cryptocurrency.trader.order_book import get_order_book_trigger
from cryptocurrency.trader.trade import trade
import time
import pandas as pd

ssh = Ssh(input_log=input_log, output_log_screened=output_log_screened, keys_file=keys_file)

def choose_to_asset(blacklist, from_asset, to_asset, latest_asset, output_log_screened=output_log_screened):
    tradable_pairs = ssh.get_logs_from_server(server_log=ssh.output_log_screened)
    if tradable_pairs is None:
        to_asset = sell_asset
    else:
        print('.', end='')
        tradable_pairs = tradable_pairs.sort_values(by='last_price_move', ascending=False)
        tradable_assets = list(set(tradable_pairs['symbol'].tolist()))
        if from_asset in tradable_assets:
            to_asset = latest_asset = from_asset
        else:
            is_buyable = False
            for test_asset in tradable_assets:
                test_pair = select_pair_with_highest_quote_volume_from_base_asset(
                    base_asset=test_asset, conversion_table=conversion_table, exchange_info=exchange_info)
                #time.sleep(0.2)
                if check_if_asset_from_pair_is_buyable(
                    blacklist, pair, exchange_info, take_profit_count=6, stop_loss_count=3):
                    #if get_order_book_trigger(client=client, symbol=test_pair, threshold=10000):
                    latest_asset = test_asset
                    is_buyable = True
                    break
            to_asset = latest_asset if is_buyable else sell_asset
    return latest_asset, to_asset

def trade_conditionally(blacklist, client, exchange_info, to_asset):
    #conversion_table = ssh.get_logs_from_server(server_log=ssh.input_log)
    conversion_table = get_conversion_table(client=client, 
                                            exchange_info=exchange_info, 
                                            offset_s=offset_s, 
                                            dump_raw=False, 
                                            as_pair=True, 
                                            minimal=False, 
                                            extra_minimal=False, 
                                            super_extra_minimal=False, 
                                            convert_to_USDT=False)
    from_asset, converted_quantity, quantity, priority = \
        select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                         exchange_info=exchange_info)
    request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                    exchange_info=exchange_info, priority=priority)
    if request is not None:
        if to_asset != sell_asset:
            pair = select_pair_with_highest_quote_volume_from_base_asset(
                to_asset, conversion_table, exchange_info)
            blacklist = add_entry_to_blacklist(blacklist, pair, conversion_table, exchange_info, reason=None)
            base_asset_from_pair = exchange_info[exchange_info['symbol'] == pair]['base_asset'].iat[0]
            pair = blacklist[blacklist['base_asset'] == base_asset_from_pair]['symbol'].iat[0]
            blacklist.loc[blacklist['symbol'] == pair,'symbol'] = request['symbol']
            blacklist.loc[blacklist['symbol'] == pair,'close'] = float(request['fills'][0]['price'])
        from_asset = to_asset
    return blacklist, from_asset, to_asset

latest_asset = from_asset
price_when_bought = quantity

blacklist = pd.DataFrame(columns=['symbol', 'base_asset', 'close', 
                                  'take_profit_count', 'stop_loss_count'])

while True:
    latest_asset, to_asset = choose_to_asset(
        blacklist, from_asset, to_asset, latest_asset, output_log_screened=output_log_screened)
    if from_asset != to_asset:
        blacklist, from_asset, to_asset = trade_conditionally(
            blacklist=blacklist, client=client, exchange_info=exchange_info, to_asset=to_asset)
    elif from_asset != sell_asset:
        #conversion_table = ssh.get_logs_from_server(server_log=ssh.input_log)
        conversion_table = get_conversion_table(
            client=client, exchange_info=exchange_info, offset_s=offset_s, dump_raw=False, as_pair=True, 
            minimal=False, extra_minimal=False, super_extra_minimal=False, convert_to_USDT=False)
        blacklist, to_asset = check_take_profit_and_stop_loss(
            blacklist, from_asset, to_asset, conversion_table, exchange_info, 
            latest_asset, take_profit=take_profit, stop_loss=stop_loss)
    blacklist = remove_older_entries_in_blacklist(blacklist, frequency='15min')
    time.sleep(2)

ssh.close()